In [13]:
import os
# suppress silly log messages from tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import importlib
import tensorflow as tf
import numpy as np
import random
import structure_prediction_utils as utils
from Autoencoder import AutoEncoder
# import resnet_model as res
import res_autoencoder as res
import vit_model as vit
importlib.reload(res)
from tensorflow import keras

In [14]:
# import os
# import csv
# # import time

# def save_to_file(avg_loss_list_epochs, avg_mse_loss_list_epochs, validate_loss_list_epochs):
#     # 创建新的文件夹以存储损失记录
#     timestamp = time.strftime("%Y%m%d-%H%M%S")
#     folder_name = 'each_epochs_loss'
#     os.makedirs(folder_name, exist_ok=True)

#     # 创建文件路径
#     file_path = os.path.join(folder_name, f'each_epochs_loss_{timestamp}.csv')

#     # 将损失值保存到 CSV 文件中
#     with open(file_path, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         # 写入表头
#         writer.writerow(['Epoch', 'Avg Loss', 'Avg MSE Loss', 'Validation Loss'])
        
#         # 写入每一轮的损失值
#         for i in range(len(avg_loss_list_epochs)):
#             writer.writerow([i + 1, avg_loss_list_epochs[i], avg_mse_loss_list_epochs[i], validate_loss_list_epochs[i]])

#     print(f"Loss records saved to '{file_path}'")

# # 示例调用
# avg_loss_list_epochs = [0.1, 0.2, 0.3]  # 示例数据
# avg_mse_loss_list_epochs = [0.05, 0.1, 0.15]
# validate_loss_list_epochs = [0.08, 0.18, 0.25]
# save_to_file(avg_loss_list_epochs, avg_mse_loss_list_epochs, validate_loss_list_epochs)


In [15]:
data_folder = './'
training_records = utils.load_preprocessed_data(data_folder, 'training.tfr')
validate_records = utils.load_preprocessed_data(data_folder, 'validation.tfr')
test_records = utils.load_preprocessed_data(data_folder, 'testing.tfr')

evolutionary shape= (5376,)
id shape= (1,)
primary shape= (256,)
tertiary shape= (256, 3)
primary_onehot shape= (256, 21)
mask shape= (256,)
true_distances shape= (256, 256)
distance_mask shape= (256, 256)
Value info of one data record, structure: tf.Tensor([1806.5  322.7 6882.9], shape=(3,), dtype=float32)
evolutionary shape= (5376,)
id shape= (1,)
primary shape= (256,)
tertiary shape= (256, 3)
primary_onehot shape= (256, 21)
mask shape= (256,)
true_distances shape= (256, 256)
distance_mask shape= (256, 256)
Value info of one data record, structure: tf.Tensor([0. 0. 0.], shape=(3,), dtype=float32)
evolutionary shape= (5376,)
id shape= (1,)
primary shape= (256,)
tertiary shape= (256, 3)
primary_onehot shape= (256, 21)
mask shape= (256,)
true_distances shape= (256, 256)
distance_mask shape= (256, 256)
Value info of one data record, structure: tf.Tensor([0. 0. 0.], shape=(3,), dtype=float32)


In [16]:
class ProteinStructurePredictor0(keras.Model):
    def __init__(self):
        super().__init__()
        self.layer0 = keras.layers.Conv2D(5, 5, activation='gelu', padding="same")
        self.layer1 = keras.layers.Conv2D(1, 1, activation='gelu', padding="same")

    #@tf.function
    def call(self, inputs, mask=None):
        print("start call", '-'*20)
        primary_one_hot = inputs['primary_onehot']
        print(f"inputs x shape is: {primary_one_hot.shape}")
        # outer sum to get a NUM_RESIDUES x NUM_RESIDUES x embedding size
        x = tf.expand_dims(primary_one_hot, -2) + tf.expand_dims(primary_one_hot, -3)
        print(f"expand_dims x shape is: {x.shape}")

        # filter the initial representation into an embedded representation
        x = self.layer0(x)
        print(f"filter layer0 x shape is: {x.shape}")


        # add positional distance information
        r = tf.range(0, utils.NUM_RESIDUES, dtype=tf.float32)
        distances = tf.abs(tf.expand_dims(r, -1) - tf.expand_dims(r, -2))
        distances_bc = tf.expand_dims(
            tf.broadcast_to(distances, [primary_one_hot.shape[0], utils.NUM_RESIDUES, utils.NUM_RESIDUES]), -1)
        print(f"distances_bc shape is: {distances_bc.shape}")
        print(f"list of shape is {x.shape}  ,  {tf.shape(x * distances_bc)}  ,   {distances_bc.shape}")
        x = tf.concat([x, x * distances_bc, distances_bc], axis=-1)
        print(f"concat result x is {x.shape}")
        # x = distances_bc
        # generate result
        x = self.layer1(x)
        print(f"filter layer1 x shape is: {x.shape}")
        print("End call", '-'*20)
        return x

In [17]:
class ProteinStructurePredictor1(keras.Model):
    def __init__(self):
        super().__init__()
        self.layer0 = keras.layers.Conv2D(5, 5, activation='gelu', padding="same")
        self.layer1 = keras.layers.Conv2D(1, 1, activation='gelu', padding="same")
        self.pooling = keras.layers.MaxPooling2D(pool_size=(8, 8))
        self.upsampling = keras.layers.UpSampling2D(size=(8, 8))
        self.attention = keras.layers.MultiHeadAttention(num_heads=1, key_dim=2)
        self.dense = keras.layers.Dense(64, activation='gelu')

    #@tf.function
    def call(self, inputs, mask=None):
        print("start call", '-'*20)
        primary_one_hot = inputs['primary_onehot']
        print(f"inputs x shape is: {primary_one_hot.shape}")
        # outer sum to get a NUM_RESIDUES x NUM_RESIDUES x embedding size
        x = tf.expand_dims(primary_one_hot, -2) + tf.expand_dims(primary_one_hot, -3)
        print(f"expand_dims x shape is: {x.shape}")

        # filter the initial representation into an embedded representation
        x = self.layer0(x)
        print(f"filter layer0 x shape is: {x.shape}")


        # add positional distance information
        r = tf.range(0, utils.NUM_RESIDUES, dtype=tf.float32)
        distances = tf.abs(tf.expand_dims(r, -1) - tf.expand_dims(r, -2))
        distances_bc = tf.expand_dims(
            tf.broadcast_to(distances, [primary_one_hot.shape[0], utils.NUM_RESIDUES, utils.NUM_RESIDUES]), -1)
        print(f"distances_bc shape is: {distances_bc.shape}")
        print(f"list of shape is {x.shape}  ,  {tf.shape(x * distances_bc)}  ,   {distances_bc.shape}")
        x = tf.concat([x, x * distances_bc, distances_bc], axis=-1)
        print(f"concat result x is {x.shape}")
        x = self.pooling(x)
        print(f'after pooling x shape is {x.shape}')
        attention_output = self.attention(x, x, x)
        print(f'attention_output shape is {attention_output.shape}')
        x = x + attention_output
        print(f"x + attention shape is {x.shape}")
        x = self.dense(x)
        print(f"x dense shape is {x.shape}")
        # x = distances_bc
        x = self.upsampling(x)
        print(f"after upsampling, x shape is: {x.shape}")
        # generate result
        x = self.layer1(x)
        print(f"filter layer1 x shape is: {x.shape}")
        print("End call", '-'*20)

        return x

In [18]:
class ProteinStructurePredictor2(keras.Model):
    def __init__(self):
        super().__init__()
        self.layer0 = keras.layers.Conv2D(5, 5, activation='gelu', padding="same")
        self.layer1 = keras.layers.Conv2D(1, 1, activation='gelu', padding="same")
        self.pooling = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.upsampling = keras.layers.UpSampling2D(size=(2, 2))
        self.attention = keras.layers.MultiHeadAttention(num_heads=4, key_dim=16)
        self.dense = keras.layers.Dense(64, activation='gelu')

    #@tf.function
    def call(self, inputs, mask=None):
        print("start call", '-'*20)
        primary_one_hot = inputs['primary_onehot']
        print(f'primary_one_hot shape is {primary_one_hot.shape}')
        attention_output = self.attention(primary_one_hot, primary_one_hot, primary_one_hot)
        print(f'attention_output shape is {attention_output.shape}')
        x = primary_one_hot + attention_output
        print(f"x + attention shape is {x.shape}")
        # outer sum to get a NUM_RESIDUES x NUM_RESIDUES x embedding size
        x = tf.expand_dims(primary_one_hot, -2) + tf.expand_dims(primary_one_hot, -3)
        print(f"expand_dims x shape is: {x.shape}")

        # filter the initial representation into an embedded representation
        x = self.layer0(x)
        print(f"filter layer0 x shape is: {x.shape}")


        # add positional distance information
        r = tf.range(0, utils.NUM_RESIDUES, dtype=tf.float32)
        distances = tf.abs(tf.expand_dims(r, -1) - tf.expand_dims(r, -2))
        distances_bc = tf.expand_dims(
            tf.broadcast_to(distances, [primary_one_hot.shape[0], utils.NUM_RESIDUES, utils.NUM_RESIDUES]), -1)
        print(f"distances_bc shape is: {distances_bc.shape}")
        print(f"list of shape is {x.shape}  ,  {tf.shape(x * distances_bc)}  ,   {distances_bc.shape}")
        x = tf.concat([x, x * distances_bc, distances_bc], axis=-1)
        print(f"concat result x is {x.shape}")
        x = self.pooling(x)
        print(f'after pooling x shape is {x.shape}')
        x = self.dense(x)
        print(f"x dense shape is {x.shape}")
        # x = distances_bc
        x = self.upsampling(x)
        print(f"after upsampling, x shape is: {x.shape}")
        # generate result
        x = self.layer1(x)
        print(f"filter layer1 x shape is: {x.shape}")
        print("End call", '-'*20)

        return x

In [19]:
class ProteinStructurePredictor4(keras.Model):
    def __init__(self):
        super().__init__()
        self.layer0 = keras.layers.Conv2D(5, 5, activation='gelu', padding="same")
        self.layer1 = keras.layers.Conv2D(1, 1, activation='gelu', padding="same")
        self.pooling = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.upsampling = keras.layers.UpSampling2D(size=(2, 2))
        self.attention = keras.layers.MultiHeadAttention(num_heads=4, key_dim=16)
        self.dense = keras.layers.Dense(64, activation='gelu')
        # self.encoder = Encoder()
        # self.decoder = Decoder()
        self.autencode = AutoEncoder()
        self.resnet = res.resnet50()

    #@tf.function
    def call(self, inputs, mask=None):
        print("start call", '-'*20)
        primary_one_hot = inputs['primary_onehot']
        print(f'primary_one_hot shape is {primary_one_hot.shape}')
        attention_output = self.attention(primary_one_hot, primary_one_hot, primary_one_hot)
        print(f'attention_output shape is {attention_output.shape}')
        x = primary_one_hot + attention_output
        print(f"x + attention shape is {x.shape}")
        # outer sum to get a NUM_RESIDUES x NUM_RESIDUES x embedding size
        x = tf.expand_dims(primary_one_hot, -2) + tf.expand_dims(primary_one_hot, -3)
        print(f"expand_dims x shape is: {x.shape}")

        x = self.resnet(x)
        print(f"After resnet x shape is: {x.shape}")
        # filter the initial representation into an embedded representation
        x = self.layer0(x)
        print(f"filter layer0 x shape is: {x.shape}")


        # add positional distance information
        r = tf.range(0, utils.NUM_RESIDUES, dtype=tf.float32)
        distances = tf.abs(tf.expand_dims(r, -1) - tf.expand_dims(r, -2))
        distances_bc = tf.expand_dims(
            tf.broadcast_to(distances, [primary_one_hot.shape[0], utils.NUM_RESIDUES, utils.NUM_RESIDUES]), -1)
        print(f"distances_bc shape is: {distances_bc.shape}")
        print(f"list of shape is {x.shape}  ,  {tf.shape(x * distances_bc)}  ,   {distances_bc.shape}")
        x = tf.concat([x, x * distances_bc, distances_bc], axis=-1)
        print(f"concat result x is {x.shape}")
        x = self.pooling(x)
        print(f'after pooling x shape is {x.shape}')
        x = self.dense(x)
        print(f"x dense shape is {x.shape}")
        # x = self.encoder(x)
        # print(f'after encoder shape is {x.shape}')
        # x = self.decoder(x)
        # print(f'after decoder shape is {x.shape}')
        x = self.autencode(x)
        print(f'autencode shape is {x.shape}')
        # x = distances_bc
        x = self.upsampling(x)
        print(f"after upsampling, x shape is: {x.shape}")
        # generate result
        x = self.layer1(x)
        print(f"filter layer1 x shape is: {x.shape}")
        print("End call", '-'*20)

        return x

In [20]:
class ProteinStructurePredictor5(keras.Model):
    def __init__(self):
        super().__init__()
        self.layer0 = keras.layers.Conv2D(7, 5, activation='gelu', padding="same")
        self.layer1 = keras.layers.Conv2D(1, 1, activation='gelu', padding="same")
        self.attention = keras.layers.MultiHeadAttention(num_heads=4, key_dim=16)
        self.dense = keras.layers.Dense(64, activation='gelu')
        self.resnet = res.resnet34()
        self.add = keras.layers.Add()

    #@tf.function
    def call(self, inputs, mask=None):
        print(f"mask shape is {mask.shape}")
        print("start call", '-'*20)
        primary_one_hot = inputs['primary_onehot']
        print(f'primary_one_hot shape is {primary_one_hot.shape}')
        attention_output = self.attention(primary_one_hot, primary_one_hot, primary_one_hot)
        print(f'attention_output shape is {attention_output.shape}')
        x = self.add([primary_one_hot , attention_output])
        print(f"x +  add attention shape is {x.shape}")
        x = self.dense(x)
        print(f"x dense shape is {x.shape}")
        # outer sum to get a NUM_RESIDUES x NUM_RESIDUES x embedding size
        x = tf.expand_dims(x, -2) + tf.expand_dims(x, -3)
        print(f"expand_dims x shape is: {x.shape}")
        # filter the initial representation into an embedded representation
        x = self.layer0(x)
        print(f"filter layer0 x shape is: {x.shape}")
        # add positional distance information
        r = tf.range(0, utils.NUM_RESIDUES, dtype=tf.float32)
        distances = tf.abs(tf.expand_dims(r, -1) - tf.expand_dims(r, -2))
        distances_bc = tf.expand_dims(
            tf.broadcast_to(distances, [primary_one_hot.shape[0], utils.NUM_RESIDUES, utils.NUM_RESIDUES]), -1)
        print(f"distances_bc shape is: {distances_bc.shape}")
        distances_bc = self.layer1(distances_bc)
        print(f"filter layer1 x shape is: {distances_bc.shape}")
        print(f"list of shape is {x.shape}  ,  {tf.shape(x * distances_bc)}  ,   {distances_bc.shape}")
        distances_bc = distances_bc * tf.expand_dims(mask, axis=-1)
        x = tf.concat([x, x * distances_bc, distances_bc], axis=-1)
        print(f"concat result x is {x.shape}")
        x = self.resnet(x)
        print(f"After resnet x shape is: {x.shape}")
        x = self.layer1(x)
        print(f"filter layer1 x shape is: {x.shape}")
        print("End call", '-'*20)

        return x
    


In [21]:
class ProteinStructurePredictor6(keras.Model):
    def __init__(self):
        super().__init__()
        self.layer0 = keras.layers.Conv2D(7, 5, activation='gelu', padding="same")
        self.layer1 = keras.layers.Conv2D(1, 1, activation='gelu', padding="same")
        self.attention = keras.layers.MultiHeadAttention(num_heads=4, key_dim=16)
        self.dense = keras.layers.Dense(64, activation='gelu')
        self.resnet = res.resnet34()
        self.vit = vit.vit_base_patch16_224_in21k()
        self.add = keras.layers.Add()

    #@tf.function
    def call(self, inputs, mask=None):
        print(f"mask shape is {mask.shape}")
        print("start call", '-'*20)
        primary_one_hot = inputs['primary_onehot']
        print(f'primary_one_hot shape is {primary_one_hot.shape}')
        attention_output = self.attention(primary_one_hot, primary_one_hot, primary_one_hot)
        print(f'attention_output shape is {attention_output.shape}')
        x = self.add([primary_one_hot , attention_output])
        print(f"x +  add attention shape is {x.shape}")
        x = self.dense(x)
        print(f"x dense shape is {x.shape}")
        # outer sum to get a NUM_RESIDUES x NUM_RESIDUES x embedding size
        x = tf.expand_dims(x, -2) + tf.expand_dims(x, -3)
        print(f"expand_dims x shape is: {x.shape}")
        # filter the initial representation into an embedded representation
        x = self.layer0(x)
        print(f"filter layer0 x shape is: {x.shape}")
        # add positional distance information
        r = tf.range(0, utils.NUM_RESIDUES, dtype=tf.float32)
        distances = tf.abs(tf.expand_dims(r, -1) - tf.expand_dims(r, -2))
        distances_bc = tf.expand_dims(
            tf.broadcast_to(distances, [primary_one_hot.shape[0], utils.NUM_RESIDUES, utils.NUM_RESIDUES]), -1)
        print(f"distances_bc shape is: {distances_bc.shape}")
        distances_bc = self.layer1(distances_bc)
        print(f"filter layer1 x shape is: {distances_bc.shape}")
        print(f"list of shape is {x.shape}  ,  {tf.shape(x * distances_bc)}  ,   {distances_bc.shape}")
        distances_bc = distances_bc * tf.expand_dims(mask, axis=-1)
        x = tf.concat([x, x * distances_bc, distances_bc], axis=-1)
        print(f"concat result x is {x.shape}")
        resnet = self.resnet(x)
        print(f"After resnet x shape is: {resnet.shape}")
        # x = self.layer1(x)
        # print(f"filter layer1 x shape is: {x.shape}")
        vit = self.vit(x)
        print(f"self.vit x shape is: {vit.shape}")
        x = tf.expand_dims(vit, axis=-1) + resnet
        print(f"end add x shape is: {x.shape}")
        print("End call", '-'*20)

        return x
    


In [22]:
class ProteinStructurePredictor7(keras.Model):
    def __init__(self):
        super().__init__()
        self.layer0 = keras.layers.Conv2D(7, 5, activation='gelu', padding="same")
        self.layer1 = keras.layers.Conv2D(256, 16, 16,activation='gelu', padding="same")
        self.attention = keras.layers.MultiHeadAttention(num_heads=4, key_dim=16)
        self.dense = keras.layers.Dense(64, activation='gelu')
        self.resnet = res.resnet34()
        self.add = keras.layers.Add()

    #@tf.function
    def call(self, inputs, mask=None):
        print(f"mask shape is {mask.shape}")
        print("start call", '-'*20)
        primary_one_hot = inputs['primary_onehot']
        print(f'primary_one_hot shape is {primary_one_hot.shape}')
        attention_output = self.attention(primary_one_hot, primary_one_hot, primary_one_hot)
        print(f'attention_output shape is {attention_output.shape}')
        x = self.add([primary_one_hot , attention_output])
        print(f"x +  add attention shape is {x.shape}")
        x = self.dense(x)
        print(f"x dense shape is {x.shape}")
        # outer sum to get a NUM_RESIDUES x NUM_RESIDUES x embedding size
        x = tf.expand_dims(x, -2) + tf.expand_dims(x, -3)
        print(f"expand_dims x shape is: {x.shape}")
        # filter the initial representation into an embedded representation
        x = self.layer0(x)
        print(f"filter layer0 x shape is: {x.shape}")
        # add positional distance information
        r = tf.range(0, utils.NUM_RESIDUES, dtype=tf.float32)
        distances = tf.abs(tf.expand_dims(r, -1) - tf.expand_dims(r, -2))
        distances_bc = tf.expand_dims(
            tf.broadcast_to(distances, [primary_one_hot.shape[0], utils.NUM_RESIDUES, utils.NUM_RESIDUES]), -1)
        print(f"distances_bc shape is: {distances_bc.shape}")
        print(f"list of shape is {x.shape}  ,  {tf.shape(x * distances_bc)}  ,   {distances_bc.shape}")
        distances_bc = distances_bc * tf.expand_dims(mask, axis=-1)
        x = tf.concat([x, x * distances_bc, distances_bc], axis=-1)
        print(f"concat result x is {x.shape}")
        x = self.resnet(x)
        print(f"After resnet x shape is: {x.shape}")
        x = self.layer1(x)
        print(f"filter layer1 x shape is: {x.shape}")
        x = tf.reshape(x, [tf.shape(x)[0], 256, 256])
        print(f"reshape x shape is: {x.shape}")
        print("End call", '-'*20)

        return x
    


In [26]:
model = ProteinStructurePredictor6()
model.optimizer = keras.optimizers.Adam(learning_rate=1e-2)
model.batch_size = 4
epochs = 5
def get_n_records(batch):
    return batch['primary_onehot'].shape[0]
def get_input_output_masks(batch):
    inputs = {'primary_onehot':batch['primary_onehot']}
    outputs = batch['true_distances']
    masks = batch['distance_mask']
    return inputs, outputs, masks

In [27]:
epoch_training_records = training_records.shuffle(buffer_size=256).batch(model.batch_size, drop_remainder=False)
train_loss=utils.mse_loss
for batch in epoch_training_records:
    inputs, labels, masks = get_input_output_masks(batch)
    print(f'tf.reduce_sum(mask): {tf.reduce_sum(masks[0])}')
    print(f'tf.reduce_sum(mask): {tf.reduce_sum(masks[1])}')
    print(f'tf.reduce_sum(mask): {tf.reduce_sum(masks[2])}')
    print(f'tf.reduce_sum(mask): {tf.reduce_sum(masks[3])}')
    with tf.GradientTape() as tape:
        print(inputs['primary_onehot'].shape,labels.shape,masks.shape)
        outputs = model(inputs, masks)
        l = train_loss(outputs, labels, masks)
        batch_loss = tf.reduce_sum(l)
        break

tf.reduce_sum(mask): 65536.0
tf.reduce_sum(mask): 64009.0
tf.reduce_sum(mask): 65536.0
tf.reduce_sum(mask): 65536.0
(4, 256, 21) (4, 256, 256) (4, 256, 256)
mask shape is (4, 256, 256)
start call --------------------
primary_one_hot shape is (4, 256, 21)
attention_output shape is (4, 256, 21)
x +  add attention shape is (4, 256, 21)
x dense shape is (4, 256, 64)
expand_dims x shape is: (4, 256, 256, 64)
filter layer0 x shape is: (4, 256, 256, 7)
distances_bc shape is: (4, 256, 256, 1)
filter layer1 x shape is: (4, 256, 256, 1)
list of shape is (4, 256, 256, 7)  ,  [  4 256 256   7]  ,   (4, 256, 256, 1)
concat result x is (4, 256, 256, 15)


ValueError: Input 0 is incompatible with layer ResNet_autoencoder: expected shape=(None, 256, 256, 8), found shape=(4, 256, 256, 15)

In [13]:
import tensorflow as tf

# 查看所有物理 GPU 设备
gpus = tf.config.list_physical_devices('GPU')
print("Available GPUs:", gpus)


Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
